In [48]:
import pandas as pd
import numpy as np
import os
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# Ключевая ставка

In [250]:

df_key_rate = pd.read_csv('./data/data_183.csv', sep=';')
df_key_rate.head()
# keep only first day of month where date has format yyyy-mm-dd
df_key_rate = df_key_rate[df_key_rate['date'].str.endswith('-01')]
# only start from 2009 year
df_key_rate = df_key_rate[df_key_rate['date'] >= '2009-01-01']
# save to folder processed data
# new column where key_rate if refinance rate is NaN
df_key_rate['valuesmy'] = df_key_rate['key_rate'].fillna(df_key_rate['refinancing_rate'])
# drop key_rate and refinancing_rate
df_key_rate = df_key_rate.drop(['key_rate', 'refinancing_rate'], axis=1)
# sort and index fales
df_key_rate = df_key_rate.sort_values(by=['date']).reset_index(drop=True)
# convert to float
df_key_rate['valuesmy'] = df_key_rate['valuesmy'].astype(float)
df_key_rate.to_csv('./processed_data/key_rate.csv', index=False)
df_key_rate.head()


,date,valuesmy
0,2009-01-01,13.0
1,2009-02-01,13.0
2,2009-03-01,13.0
3,2009-04-01,13.0
4,2009-05-01,12.5


In [203]:
df_key_rate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127 entries, 6210 to 10217
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              127 non-null    object 
 1   key_rate          58 non-null     float64
 2   refinancing_rate  84 non-null     float64
 3   values            127 non-null    float64
dtypes: float64(3), object(1)
memory usage: 5.0+ KB


## Просроченная задолженность по кредитам, предоставленным юридическим лицам — резидентам и индивидуальным предпринимателям в рублях, по видам экономической деятельности и отдельным направлениям использования средств


In [234]:
df_base = pd.read_csv('./data/302-09.csv')
df_base["Отчетная дата"] = pd.to_datetime(df_base["Отчетная дата"], format='%m/%d/%Y')
# add new rows 2009-01-01, 2009-02-01, 2009-03-01,  with the same values as 2009-04-01
df_base = df_base.append([df_base.iloc[0]]*3, ignore_index=True)
df_base = df_base.sort_values(by=['Отчетная дата'])
df_base.iloc[0:3, 0] = pd.to_datetime(['2009-01-01', '2009-02-01', '2009-03-01'])
df_base = df_base.sort_values(by=['Отчетная дата'], ignore_index=True)
# sort by date
# replace all columns exept Отчетная дата str " " to "" and float
df_base.iloc[:, 1:] = df_base.iloc[:, 1:].replace(' ', '', regex=True).astype(float)
df_base.to_csv('./processed_data/df_base.csv', index=False)
df_base.head(6)

C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\3882353623.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_base = df_base.append([df_base.iloc[0]]*3, ignore_index=True)


,Отчетная дата,Всего,добыча полезных ископаемых,добыча топливно-энергетических полезных ископаемых,обрабатывающие производства,производство пищевых продуктов,обработка древесины и производство изделий из дерева,целлюлозно-бумажное производство; издательская и полиграфическая деятельность,"производство кокса, нефтепродуктов и ядерных материалов",химическое производство,...,сельское хозяйство охота и лесное хозяйство,"сельское хозяйство, охота и предоставление услуг в этих областях",строительство,строительство зданий и сооружений,транспорт и связь,"деятельность воздушного транспорта, подчиняющегося и не подчиняющегося расписанию","оптовая и розничная торговля; ремонт автотранспортных средств, мотоциклов, бытовых изделий и предметов личного пользования","операции с недвижимым имуществом, аренда и предоставление услуг",прочие виды деятельности,на завершение расчетов
0,2009-01-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,15256.0,10854.0,43447.0,33272.0,10279.0,3181.0,138637.0,18999.0,30438.0,13062.0
1,2009-02-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,15256.0,10854.0,43447.0,33272.0,10279.0,3181.0,138637.0,18999.0,30438.0,13062.0
2,2009-03-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,15256.0,10854.0,43447.0,33272.0,10279.0,3181.0,138637.0,18999.0,30438.0,13062.0
3,2009-04-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,15256.0,10854.0,43447.0,33272.0,10279.0,3181.0,138637.0,18999.0,30438.0,13062.0
4,2009-05-01,380065.0,4032.0,1871.0,74435.0,24133.0,3711.0,1999.0,672.0,2615.0,...,15318.0,12145.0,45735.0,35817.0,11171.0,2853.0,153720.0,23304.0,34319.0,14346.0
5,2009-06-01,409667.0,3235.0,1967.0,80603.0,24110.0,3984.0,2156.0,672.0,2989.0,...,16065.0,12997.0,54593.0,43278.0,11626.0,3251.0,164652.0,25073.0,34600.0,14857.0


# Индекс потребительских цен

In [262]:

df_index_price = pd.read_csv('./data/index_price.csv')
df_index_price.head()
# replace df_index_price["Дата"] 
# "декабрь 2022 г." to  12/1/2022
# "ноябрь  2022 г." to  11/1/2022
# e.t.c
# remove space in the end of the string
df_index_price["Дата"] = df_index_price["Дата"].str.replace("декабрь", "12/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("ноябрь", "11/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("октябрь", "10/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("сентябрь", "9/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("август", "8/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("июль", "7/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("июнь", "6/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("май", "5/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("апрель", "4/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("март", "3/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("февраль", "2/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("январь", "1/1")
df_index_price["Дата"] = df_index_price["Дата"].str.replace(" ", "/")
df_index_price["Дата"] = df_index_price["Дата"].str.replace("/г.", "")
# remove last slash
df_index_price["Дата"] = pd.to_datetime(df_index_price["Дата"], format='%m/%d/%Y')
# sort by date
df_index_price = df_index_price.sort_values(by=['Дата'], ignore_index=True)
# save
#convert to float
# replace "-" to 0
df_index_price = df_index_price.replace('-', 0)
df_index_price = df_index_price.replace('- ', 0)
df_index_price.iloc[:, 1:] = df_index_price.iloc[:, 1:].astype(float)
df_index_price.to_csv('./processed_data/index_price.csv', index=False)
df_index_price.head()
df_index_price_mod = df_index_price.copy()
# column name Дата, prev_year, pred_month, pred_month_year
df_index_price_mod.columns = ['date', 'prev_year', 'pred_month', 'pred_month_year']

df_index_price_mod.head()


C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\2357132673.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_index_price["Дата"] = df_index_price["Дата"].str.replace("/г.", "")


,date,prev_year,pred_month,pred_month_year
0,2009-01-01,0.0,102.4,113.4
1,2009-02-01,104.1,101.7,0.0
2,2009-03-01,105.4,101.3,0.0
3,2009-04-01,106.2,100.7,0.0
4,2009-05-01,106.8,100.6,0.0


# ВВП Российской Федерации по годам

In [241]:

df_vvp_year = pd.read_csv('./data/vvp_year.csv')
df_vvp_year.head()
# extend table for every month with repeat the same value as year
# duplicate each year 12 times
df_vvp_year = df_vvp_year.reindex(df_vvp_year.index.repeat(12))
df_vvp_year = df_vvp_year.reset_index(drop=True)
# add month column
df_vvp_year['month'] = np.tile(np.arange(1, 13), len(df_vvp_year) // 12)
# add date column
df_vvp_year["year"]= df_vvp_year["Год"].astype(str)
df_vvp_year['date'] = pd.to_datetime(df_vvp_year[['year', 'month']].assign(DAY=1))
# remove year and month columns
df_vvp_year = df_vvp_year.drop(['Год', 'month', 'year'], axis=1)
# put date on the first
cols = df_vvp_year.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_vvp_year = df_vvp_year[cols]
# sort
df_vvp_year = df_vvp_year.sort_values(by=['date'], ignore_index=True)
# convert to float 
# replace " " to ""
df_vvp_year.iloc[:, 1:] = df_vvp_year.iloc[:, 1:].replace(' ', '', regex=True)
df_vvp_year.iloc[:, 1:] = df_vvp_year.iloc[:, 1:].astype(float)
df_vvp_year.to_csv('./processed_data/vvp_year.csv', index=False)
df_vvp_year.head(2)

,date,"ВВП в текущих ценах, млрд руб.","Рост ВВП (к предыдущему году), %","ВВП по ППС, в ценах 2018, млрд дол.","ВВП в текущих долларах, млрд","Индекс-дефлятор, %"
0,2009-01-01,38807.2,-7.8,3054.26,1309.17,102.0
1,2009-02-01,38807.2,-7.8,3054.26,1309.17,102.0


## only $$$

In [127]:

df_bucks = pd.read_csv("./data/RC_F01_01_2009_T01_03_2023.csv")
df_bucks.head()
# sort by date and to datetime
df_bucks["data"] = pd.to_datetime(df_bucks["data"], format='%m/%d/%Y')
df_bucks.head()
df_bucks = df_bucks[df_bucks['data'].dt.day == 1]
df_bucks = df_bucks.sort_values(by=['data'], ignore_index=True)
df_bucks.head()

# create dataframe with data from 2009 to 2023 with step 1 month
df_data = pd.DataFrame(pd.date_range(start='2009-01-01', end='2023-02-01', freq='MS'), columns=['date'])
# merge with df_bucks
df_data = pd.merge(df_data, df_bucks, how='left', left_on='date', right_on='data')

# fill NaN with the same values of columns as previous
df_data = df_data.fillna(method='ffill')
# to csv
df_data = df_data.drop(['data'], axis=1)

df_data.to_csv('./processed_data/bucks_full_only.csv', index=False)
# drop data
df_data.head()


,date,nominal,curs,cdx
0,2009-01-01,1.0,29.3916,Доллар США
1,2009-02-01,1.0,29.3916,Доллар США
2,2009-03-01,1.0,29.3916,Доллар США
3,2009-04-01,1.0,33.9032,Доллар США
4,2009-05-01,1.0,32.9740,Доллар США


# Unemployment

In [151]:
# create dataframe with quarters
df_quarter = pd.DataFrame(pd.date_range(start='2009-01-01', end='2021-12-31', freq='MS'), columns=['date'])
df_quarter.head()
df_unemploy = pd.read_csv("./data/unemploy.csv", sep='\t')
# repeat each values 3 times
df_unemploy = df_unemploy.reindex(df_unemploy.index.repeat(3))
df_unemploy = df_unemploy.reset_index(drop=True)
# add columns Значение from df_unemploy to df_quarter
df_quarter["Values"] = df_unemploy["Значение"]
# save to csv
df_quarter.to_csv('./processed_data/unemploy_rate.csv', index=False)
df_quarter.head(20)


,date,Values
0,2009-01-01,8.3
1,2009-02-01,8.3
2,2009-03-01,8.3
3,2009-04-01,8.3
4,2009-05-01,8.3
5,2009-06-01,8.3
6,2009-07-01,8.3
7,2009-08-01,8.3
8,2009-09-01,8.3
9,2009-10-01,8.3


## Inflation Rate

In [251]:
df_inflation = pd.read_csv("./data/russia.inflation.monthly.csv", sep=',')
df_inflation.head()
# change format Год, Январь, Февраль, Март, Апрель, Май, Июнь, Июль, Август, Сентябрь, Октябрь, Ноябрь, Декабрь,  to
# 2009-01-01 Всего
# 2009-02-01 Всего
# e.t.c
#drop Всего
df_inflation = df_inflation.drop([' Всего'], axis=1)
df_inflation = df_inflation.melt(id_vars=['Год'], var_name='month', value_name='inflation')
df_inflation.head()
# change Год month to date
df_inflation["month"] = df_inflation["month"].str.replace(" Январь", "1/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Февраль", "2/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Март", "3/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Апрель", "4/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Май", "5/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Июнь", "6/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Июль", "7/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Август", "8/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Сентябрь", "9/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Октябрь", "10/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Ноябрь", "11/1")
df_inflation["month"] = df_inflation["month"].str.replace(" Декабрь", "12/1")
df_inflation.head()
# combine Год and month
df_inflation['date'] = df_inflation["Год"].astype(str) +"/"+ df_inflation["month"].astype(str)
# to datetime
df_inflation["date"] = pd.to_datetime(df_inflation["date"], format='%Y/%m/%d')
# drop Год and month
df_inflation = df_inflation.drop(['Год', 'month'], axis=1)
# sort
# only 2009-2023
df_inflation = df_inflation[df_inflation['date'].dt.year >= 2009]
df_inflation = df_inflation.sort_values(by=['date'], ignore_index=True)

df_inflation.to_csv('./processed_data/inflation.csv', index=False)
df_inflation.head()


,inflation,date
0,2.37,2009-01-01
1,1.65,2009-02-01
2,1.31,2009-03-01
3,0.69,2009-04-01
4,0.57,2009-05-01


# Merge all into 1 table

In [263]:
# from 2009-01-01 to 2019-01-01
df_base["inflation_rate"] = df_inflation["inflation"]
df_base["unemploy_rate"] = df_quarter["Values"]
df_base["bucks_rate"] = df_data["curs"]
df_base["gdp"] = df_vvp_year["ВВП в текущих ценах, млрд руб."]
df_base["gdp_growth"] = df_vvp_year["Рост ВВП (к предыдущему году), %"]
df_base["gdp_pps"] = df_vvp_year["ВВП по ППС, в ценах 2018, млрд дол."]
df_base["gdp_usd"] = df_vvp_year["ВВП в текущих долларах, млрд"]
df_base["gdp_deflator"] = df_vvp_year["Индекс-дефлятор, %"]
# df_index_price_mod prev_year	pred_month	pred_month_year
df_base["index_price_mod"] = df_index_price_mod["pred_month_year"]
df_base["index_price_mod_growth"] = df_index_price_mod["pred_month"]
df_base["index_price_mod_growth_prev_year"] = df_index_price_mod["prev_year"]
# fill "-" with 0
df_base = df_base.replace('-', 0)
df_base['key_rate'] = df_key_rate['valuesmy']
# save to csv
df_base.to_csv('./processed_data/all_features_without_swan.csv', index=False)
df_base.head()


,Отчетная дата,Всего,добыча полезных ископаемых,добыча топливно-энергетических полезных ископаемых,обрабатывающие производства,производство пищевых продуктов,обработка древесины и производство изделий из дерева,целлюлозно-бумажное производство; издательская и полиграфическая деятельность,"производство кокса, нефтепродуктов и ядерных материалов",химическое производство,...,gdp_usd,gdp_deflator,index_price_mod,index_price_mod_growth,index_price_mod_growth_prev_year,key_rate,is_swan_event_covid,is_swan_event_default,is_swan_event_oil,is_swan_event_crimea
0,2009-01-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,1309.17,102.0,113.4,102.4,0.0,13.0,0,1,0,0
1,2009-02-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,1309.17,102.0,0.0,101.7,104.1,13.0,0,1,0,0
2,2009-03-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,1309.17,102.0,0.0,101.3,105.4,13.0,0,1,0,0
3,2009-04-01,346769.0,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,...,1309.17,102.0,0.0,100.7,106.2,13.0,0,1,0,0
4,2009-05-01,380065.0,4032.0,1871.0,74435.0,24133.0,3711.0,1999.0,672.0,2615.0,...,1309.17,102.0,0.0,100.6,106.8,12.5,0,1,0,0


## Black Swan

In [264]:
# covid was from 2020-01-30 to 2021-04-01 
df_base["is_swan_event_covid"] = 0
df_base.loc[(df_base['Отчетная дата'] >= '2020-01-30') & (df_base['Отчетная дата'] <= '2021-04-01'), 'is_swan_event_covid'] = 1

# default was from 2009-01-01 to 2010-01-01
df_base["is_swan_event_default"] = 0
df_base.loc[(df_base['Отчетная дата'] >= '2009-01-01') & (df_base['Отчетная дата'] <= '2010-01-01'), 'is_swan_event_default'] = 1

# pick oil price from 2010-01-01 to 2011-04-01
df_base["is_swan_event_oil"] = 0
df_base.loc[(df_base['Отчетная дата'] >= '2010-01-01') & (df_base['Отчетная дата'] <= '2011-04-01'), 'is_swan_event_oil'] = 1

# crimea event
df_base["is_swan_event_crimea"] = 0
df_base.loc[(df_base['Отчетная дата'] >= '2014-03-01') & (df_base['Отчетная дата'] <= '2014-04-01'), 'is_swan_event_crimea'] = 1

# save to csv
df_base.to_csv('./processed_data/all_features_with_swan.csv', index=False)

In [ ]:
from sklearn.preprocessing import StandardScaler

# normalize all columns exept data

## Catboost

In [335]:


df_train = df_base[df_base['Отчетная дата'] < '2018-01-01']
print(df_train.shape)
df_test = df_base[df_base['Отчетная дата'] >= '2018-01-01']
print(df_test.shape)
# add mean_5, mean_10, mean_15 "Всего"
def add_rols(df):
    # to float
    # remove " " 
    df["mean_5"] = df["Всего"].rolling(5).mean()
    df["mean_10"] = df["Всего"].rolling(10).mean()
    df["mean_15"] = df["Всего"].rolling(15).mean()
    df["mean_20"] = df["Всего"].rolling(20).mean()
    df["mean_25"] = df["Всего"].rolling(25).mean()
    # add std
    df["std_5"] = df["Всего"].rolling(5).std()
    df["std_10"] = df["Всего"].rolling(10).std()
    df["std_15"] = df["Всего"].rolling(15).std()
    df["std_20"] = df["Всего"].rolling(20).std()
    df["std_25"] = df["Всего"].rolling(25).std()
    df.fillna(0, inplace=True)
    return df

df_train = add_rols(df_train)
y = df_train["Всего"]
df_train.drop(columns=["Всего"], inplace=True)
df_train.head(2)

(108, 43)
(13, 43)


C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\2700168745.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mean_5"] = df["Всего"].rolling(5).mean()
C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\2700168745.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mean_10"] = df["Всего"].rolling(10).mean()
C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\2700168745.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,Отчетная дата,добыча полезных ископаемых,добыча топливно-энергетических полезных ископаемых,обрабатывающие производства,производство пищевых продуктов,обработка древесины и производство изделий из дерева,целлюлозно-бумажное производство; издательская и полиграфическая деятельность,"производство кокса, нефтепродуктов и ядерных материалов",химическое производство,производство прочих неметаллических минеральных продуктов,...,mean_5,mean_10,mean_15,mean_20,mean_25,std_5,std_10,std_15,std_20,std_25
0,2009-01-01,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,2758.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2009-02-01,3084.0,1813.0,69830.0,23171.0,3464.0,1861.0,381.0,2475.0,2758.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [347]:
import catboost as cb
from catboost import CatBoostRegressor as cbr
from catboost import Pool

# catboost
model = cbr(iterations=1000,
            learning_rate=0.1,  
            depth=6,
            loss_function='RMSE',
            eval_metric='RMSE',
            random_seed=42,
            )
model.fit(df_train, y, verbose=100)

0:	learn: 458004.4675891	total: 1.68ms	remaining: 1.68s
100:	learn: 7930.3660044	total: 108ms	remaining: 959ms
200:	learn: 2558.6745089	total: 226ms	remaining: 897ms
300:	learn: 954.4054289	total: 346ms	remaining: 804ms
400:	learn: 349.2510949	total: 472ms	remaining: 705ms
500:	learn: 137.4832638	total: 576ms	remaining: 574ms
600:	learn: 55.5432746	total: 680ms	remaining: 451ms
700:	learn: 23.5070121	total: 790ms	remaining: 337ms
800:	learn: 9.6494030	total: 902ms	remaining: 224ms
900:	learn: 3.7680439	total: 1.02s	remaining: 112ms
999:	learn: 1.5175451	total: 1.14s	remaining: 0us


In [348]:
def mod(testdf):
    testdf.drop(columns=["Всего"], inplace=True)
    forecasts = y.values
    mean_5 = forecasts[-5:].mean()
    mean_10 = forecasts[-10:].mean()
    mean_15 = forecasts[-15:].mean()
    mean_20 = forecasts[-20:].mean()
    mean_25 = forecasts[-25:].mean()
    std_5 = forecasts[-5:].std()
    std_10 = forecasts[-10:].std()
    std_15 = forecasts[-15:].std()
    std_20 = forecasts[-20:].std()
    std_25 = forecasts[-25:].std()
    
    testdf["mean_5"] = mean_5
    testdf["mean_10"] = mean_10
    testdf["mean_15"] = mean_15
    testdf["mean_20"] = mean_20
    testdf["mean_25"] = mean_25
    testdf["std_5"] = std_5
    testdf["std_10"] = std_10
    testdf["std_15"] = std_15
    testdf["std_20"] = std_20
    testdf["std_25"] = std_25
    testdf.fillna(0, inplace=True)

    res = model.predict(testdf)
    return res
mod(df_test)


C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\2439481001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf.drop(columns=["Всего"], inplace=True)
C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\2439481001.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["mean_5"] = mean_5
C:\Users\Sergey\AppData\Local\Temp\ipykernel_35428\2439481001.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

array([1751561.39951837, 1748379.32478601, 1757373.19380192,
       1742595.49134375, 1732157.14352578, 1729927.60381374,
       1705776.55341838, 1740151.52428844, 1735713.77518876,
       1735059.5888828 , 1755644.98101332, 1669226.16070566,
       1635853.32088954])

In [377]:
from tqdm.notebook import tqdm
# catboost
import warnings
warnings.filterwarnings("ignore")
part_train = df_train.copy()
part_test = df_test.copy()
# drop "Всего"
# part_test.drop(columns=["Всего"], inplace=True)
forecasts = y.values
print(part_test.shape[0])
# generate dataframe with date from 2018-02-01 to 2023-04-01 only by month
dfs = pd.DataFrame(pd.date_range(start='2018-02-01', end='2024-04-01', freq='MS'), columns=["Отчетная дата"])
for i in range(1, part_test.shape[0]+50):
    local = part_test.iloc[i-1:i]
    mean_5 = forecasts[-5:].mean()
    mean_10 = forecasts[-10:].mean()
    mean_15 = forecasts[-15:].mean()
    mean_20 = forecasts[-20:].mean()
    mean_25 = forecasts[-25:].mean()
    std_5 = forecasts[-5:].std()
    std_10 = forecasts[-10:].std()
    std_15 = forecasts[-15:].std()
    std_20 = forecasts[-20:].std()
    std_25 = forecasts[-25:].std()
    
    local["mean_5"] = mean_5
    local["mean_10"] = mean_10
    local["mean_15"] = mean_15
    local["mean_20"] = mean_20
    local["mean_25"] = mean_25
    local["std_5"] = std_5
    local["std_10"] = std_10
    local["std_15"] = std_15
    local["std_20"] = std_20
    local["std_25"] = std_25
    if i >= part_test.shape[0]:
        # fill everything with last value
        local = part_test.iloc[part_test.shape[0]-1:part_test.shape[0]]
    local["Отчетная дата"] = dfs["Отчетная дата"].iloc[i]
    # drop index
    output = model.predict(local)
    forecasts = np.append(forecasts, output)
    print(dfs["Отчетная дата"].iloc[i], forecasts[-1])
    
    # df_test.loc[df_test["Номер скважины"] == num, "forecast"] = part_test["forecast"].values

13
2018-03-01 00:00:00 1751561.3995183678
2018-04-01 00:00:00 1720675.6890513995
2018-05-01 00:00:00 1733888.4422455337
2018-06-01 00:00:00 1721877.9573863945
2018-07-01 00:00:00 1720431.1821281663
2018-08-01 00:00:00 1713804.1013265066
2018-09-01 00:00:00 1691414.3972433673
2018-10-01 00:00:00 1737253.1516869618
2018-11-01 00:00:00 1739982.4976295047
2018-12-01 00:00:00 1731540.1256008646
2019-01-01 00:00:00 1752171.78876602
2019-02-01 00:00:00 1664825.0808880622
2019-03-01 00:00:00 1635853.3208895412
2019-04-01 00:00:00 1635853.3208895412
2019-05-01 00:00:00 1635853.3208895412
2019-06-01 00:00:00 1635853.3208895412
2019-07-01 00:00:00 1635853.3208895412
2019-08-01 00:00:00 1635853.3208895412
2019-09-01 00:00:00 1635853.3208895412
2019-10-01 00:00:00 1635853.3208895412
2019-11-01 00:00:00 1635853.3208895412
2019-12-01 00:00:00 1635853.3208895412
2020-01-01 00:00:00 1635853.3208895412
2020-02-01 00:00:00 1635853.3208895412
2020-03-01 00:00:00 1635853.3208895412
2020-04-01 00:00:00 1635

In [378]:
# feature importance
model.get_feature_importance(prettified=True)


,Feature Id,Importances
0,mean_20,2.906374e+01
1,строительство,1.584641e+01
2,металлургическое производство и производство г...,1.208150e+01
3,транспорт и связь,8.919459e+00
4,"операции с недвижимым имуществом, аренда и пре...",2.782159e+00
5,обрабатывающие производства,2.765716e+00
6,key_rate,2.357416e+00
7,std_5,2.235842e+00
8,обработка древесины и производство изделий из ...,2.111473e+00
9,добыча полезных ископаемых,1.664708e+00


In [ ]:
1.41421+1+3.60555+3.60555+1+1.41421+1+1+1+1

In [5]:
!pip install darts

^C


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fedot 0.6.2 requires matplotlib==3.0.2; python_version == "3.7", but you have matplotlib 3.5.3 which is incompatible.
You should consider upgrading via the 'c:\users\sergey\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


  Using cached darts-0.23.1-py3-none-any.whl (592 kB)
  Using cached pyod-1.0.7.tar.gz (147 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached holidays-0.20-py3-none-any.whl (226 kB)
  Using cached nfoursid-1.0.1-py3-none-any.whl (16 kB)
  Using cached pytorch_lightning-1.9.3-py3-none-any.whl (826 kB)
  Using cached tbats-1.1.2-py3-none-any.whl (43 kB)
  Using cached prophet-1.1.2-py3-none-win_amd64.whl (12.1 MB)
  Using cached korean_lunar_calendar-0.3.1-py3-none-any.whl (9.0 kB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached PyMeeus-0.5.12.tar.gz (5.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached hijri_converter-2.2.4-py3-none-any.whl (14 kB)
  Using cached Cython-0.29.33-py2.py3-none-any.whl (987 kB)
  Using cached cmdstanpy-1.1.0-py3-none-any.whl (83 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fedot 0.6.2 requires matplotlib==3.0.2; python_version == "3.7", but you have matplotlib 3.5.3 which is incompatible.
You should consider upgrading via the 'c:\users\sergey\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


  Using cached darts-0.23.1-py3-none-any.whl (592 kB)
  Using cached nfoursid-1.0.1-py3-none-any.whl (16 kB)
  Using cached tbats-1.1.2-py3-none-any.whl (43 kB)
  Using cached pytorch_lightning-1.9.3-py3-none-any.whl (826 kB)
  Using cached shap-0.41.0-cp37-cp37m-win_amd64.whl (435 kB)
  Using cached xarray-0.20.2-py3-none-any.whl (845 kB)
  Using cached pmdarima-2.0.2-cp37-cp37m-win_amd64.whl (568 kB)
  Using cached prophet-1.1.2-py3-none-win_amd64.whl (12.1 MB)
  Using cached pyod-1.0.7.tar.gz (147 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached statsforecast-1.5.0-py3-none-any.whl (99 kB)
  Using cached holidays-0.20-py3-none-any.whl (226 kB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached PyMeeus-0.5.12.tar.gz (5.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached hijri_converter-2.2.4-py3-none-any.whl (14 kB)
  Usi

  Using cached darts-0.23.1-py3-none-any.whl (592 kB)
  Using cached xarray-0.20.2-py3-none-any.whl (845 kB)
  Using cached pmdarima-2.0.2-cp37-cp37m-win_amd64.whl (568 kB)
  Using cached statsforecast-1.5.0-py3-none-any.whl (99 kB)
  Using cached pyod-1.0.7.tar.gz (147 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached holidays-0.20-py3-none-any.whl (226 kB)
  Using cached pytorch_lightning-1.9.3-py3-none-any.whl (826 kB)
  Using cached nfoursid-1.0.1-py3-none-any.whl (16 kB)
  Using cached shap-0.41.0-cp37-cp37m-win_amd64.whl (435 kB)
  Using cached prophet-1.1.2-py3-none-win_amd64.whl (12.1 MB)
  Using cached tbats-1.1.2-py3-none-any.whl (43 kB)
  Using cached hijri_converter-2.2.4-py3-none-any.whl (14 kB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached korean_lunar_calendar-0.3.1-py3-none-any.whl (9.0 kB)
  Using cached PyMeeus-0.5.12.tar.gz (5.8 MB)
  Preparing metadata (setup.py): start

In [5]:
from darts import TimeSeries
from darts.models import *
from darts.metrics import *
from darts.dataprocessing.transformers import Scaler
import matplotlib.pyplot as plt
from darts.models.forecasting.catboost_model import CatBoostModel
df = pd.read_csv(r'./processed_data/all_features_with_swan.csv')
df.drop(columns=['is_swan_event_covid', 'is_swan_event_default', 'is_swan_event_oil', 'is_swan_event_crimea'], inplace=True)

model_baza = CatBoostModel(lags=[-i for i in range(1, 25)],
                           output_chunk_length = 12,
                           )

scaler = Scaler()


def covariates(data: pd.DataFrame) -> list:

    true_columns = [column for column, value in zip(data.columns, data.isna().sum()) if value <= 75]
    true_columns.remove('Отчетная дата')

    train_data = []
    for column in true_columns:
        ts = TimeSeries.from_dataframe(data[['Отчетная дата', column]], time_col='Отчетная дата', fillna_value=True)
        train_data.append(scaler.fit_transform(ts))

    return train_data

cov_data = covariates(df)

model_baza.fit(series=cov_data, verbose=1)
pred_cov = model_baza.predict(n=60, series=cov_data[0])

pred_cov = scaler.inverse_transform(pred_cov)
tmp_df = pred_cov.pd_dataframe()
tmp_df.plot()

ModuleNotFoundError: No module named 'matplotlib._docstring'

In [389]:
!pip install matplotlib 

  Using cached matplotlib-3.7.0-cp39-cp39-win_amd64.whl (7.6 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\sergey\\pycharmprojects\\python3_9\\venv\\Lib\\site-packages\\matplotlib\\_c_internal_utils.cp39-win_amd64.pyd'
Check the permissions.

